In [3]:
#!pip install pandera

# **Pandas**
Pandas contains data structures and data manipulation tools designed to make data cleaning and analysis fast and convenient in Python. There are two basic types in pandas
- Series
- Dataframe

In [16]:
import numpy as np
import pandas as pd
import pandera as pa
from typing import List, Dict, Tuple, Union

## Series
A **series** is one dimensional array-like object containing a sequence of values of the same type and an associated array of data label called *index*

In [ ]:
ser = pd.Series([1, 2, 3, 4, 5]) #creating a series
schema = pa.SeriesSchema(pa.Int) #validating the input type of the series.

schema(ser)#applying the schema. If any data other than int is entered, the
            #schema will raise error

0    1
1    2
2    3
3    4
4    5
dtype: int64

In [ ]:
ser = pd.Series(['Dog', 'Cat', 'Cow', 'Goat']) #creating a series
schema = pa.SeriesSchema(pa.String) #creating schema. Only string entries allowed
schema(ser) #applying schema on the series

0     Dog
1     Cat
2     Cow
3    Goat
dtype: object

In [ ]:
ser = pd.Series([1, 'apple', 2, 'mango', 3, 'strawberry', True]) #creating a series
schema = pa.SeriesSchema(pa.Object) #will now allow mix entries to the series
schema(ser)

0             1
1         apple
2             2
3         mango
4             3
5    strawberry
6          True
dtype: object

In [ ]:
ser = pd.Series([1.0, 2.0, 3.0, 4.0, 5.0])
schema = pa.SeriesSchema(pa.Float64);
schema(ser)

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64

In [ ]:
# We can also get array representation of the series
ser.array

<PandasArray>
[1.0, 2.0, 3.0, 4.0, 5.0]
Length: 5, dtype: float64

We can also declare indexes of the series

In [ ]:
#We can also specify index of the series
ser = pd.Series({"a":10,
                 "b":20,
                 "d":30,
                 "e":40,
                 "f":50,
                 })
schema = pa.SeriesSchema(pa.Int, index=pa.Index(pa.String))
          #schema for both indexes and entries

# Validate the Series against the schema
schema(ser)


a    10
b    20
d    30
e    40
f    50
dtype: int64

Data can be passed as a dictionary

In [ ]:
ser = pd.Series({1:'Dog',
                 3: 'Cat',
                 4: 'Cow',
                 2: 'Hen',
                 5: 'Goat',
                 })
schema = pa.SeriesSchema(pa.String, index=pa.Index(pa.Int))
schema(ser)

1     Dog
3     Cat
4     Cow
2     Hen
5    Goat
dtype: object

The data and can be passed as list

In [ ]:
#another way to specify index
from typing import List

index: List[str] = ['a', 'd', 'b', 'e', 'c']
fruits: List[str] = ['mango', 'strawberry', 'pineapple', 'avocado', 'apple']

ser = pd.Series(fruits, index=index)
schema = pa.SeriesSchema(pa.String, index=pa.Index(pa.String))
schema(ser)


a         mango
d    strawberry
b     pineapple
e       avocado
c         apple
dtype: object

The index and data can also be passed using arrays

In [ ]:
#can be done with array
index: np.ndarray = np.array(['a', 'd', 'b', 'e', 'c'])
fruits: np.ndarray = np.array(['mango', 'strawberry', 'pineapple', 'avocado', 'apple'])
ser = pd.Series(fruits, index=index)
schema = pa.SeriesSchema(pa.String, index=pa.Index(pa.String))
schema(ser)

a         mango
d    strawberry
b     pineapple
e       avocado
c         apple
dtype: object

In [ ]:
display(ser['d'])
display(ser[['d', 'b', 'e']])

'strawberry'

d    strawberry
b     pineapple
e       avocado
dtype: object

We can have multiple indexes

In [ ]:
from typing import Union
index: Union[List[int],List[str]] =[[1, 1, 1, 2, 2],['a', 'd', 'b', 'e', 'c']]
fruits: List[str] = ['mango', 'strawberry', 'pineapple', 'avocado', 'apple']

ser = pd.Series(fruits, index=pd.MultiIndex.from_arrays(index))

# Define a schema for the Series
schema = pa.SeriesSchema(pa.String, index=pa.MultiIndex([pa.Index(pa.Int), pa.Index(pa.String)]))

# Validate the Series against the schema
schema(ser)

1  a         mango
   d    strawberry
   b     pineapple
2  e       avocado
   c         apple
dtype: object

In [ ]:
index: Union[List[str], List[int]] = [['BS', 'BS', 'MS', 'MS', 'BS'],[1, 2, 3, 4, 5]];
students : List[str] = ['Sam', 'Eliza', 'Jack', 'Will', 'Mary']

ser = pd.Series(students, index=pd.MultiIndex.from_arrays(index))
schema = pa.SeriesSchema(pa.String, index=pa.MultiIndex([pa.Index(pa.String),pa.Index(pa.Int)]))
schema(ser)

BS  1      Sam
    2    Eliza
MS  3     Jack
    4     Will
BS  5     Mary
dtype: object

We can specify name of the series and a name for the index


In [10]:
ser = pd.Series([1.0, 2.0, 3.0, 4.0, 5.0], name='Float Series')
schema = pa.SeriesSchema(pa.Float64);
ser.index.name='Index position'
schema(ser)

Index position
0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: Float Series, dtype: float64

We can convert the index of the series

Series can be converted to dictionary

In [12]:
ser.index = ['Bob', 'Steve', 'Jeff', 'Ryan', "Alex"]
schema(ser)

Bob      1.0
Steve    2.0
Jeff     3.0
Ryan     4.0
Alex     5.0
Name: Float Series, dtype: float64

In [ ]:
ser.to_dict()#convert series to dictionary

{0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 5.0}

Data may have **nul** or **isna** numbers that need to be removed or replaced
with some number


In [5]:
ser = pd.Series([1.0, 2.0, 3.0, 4.0, 5.0], name='Float Series')
schema = pa.SeriesSchema(pa.Float64);
schema(ser)
pd.isna(schema(ser))#check if any of the entries is not a number
pd.notna(schema(ser)) #checks if any of the entries is not a none number

0    True
1    True
2    True
3    True
4    True
Name: Float Series, dtype: bool

## DataFrame
A dataframe represents a rectangular table of data and contains an ordered, named collection of columns, each of which can be of different value type.

There are different way to create dataframe. The most common way is to use dictionary

In [19]:

data: Dict[str, List[Union[str, int]]] = {'name': ['Alice', 'Bob', 'Charlie', 'David'],
        'age': [25, 30, 22, 35]}

df = pd.DataFrame(data)

# Define a Pandera schema for the DataFrame
schema = pa.DataFrameSchema({
    'name': pa.Column(pa.String),#the first column is name with string values
    'age': pa.Column(pa.Int) #the second column is age with int values
})

# Validate the DataFrame against the schema
schema.validate(df)

,name,age
0,Alice,25
1,Bob,30
2,Charlie,22
3,David,35


In [20]:
data : Dict[str, List[Union[str, int, float]]] = {
    "state": ["Ohio", "Ohion", "Ohio", "Nevada", "Nevada", "Nevada"],
    "year": [2000, 2001, 2002, 2001, 2002, 2003],
    "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]
}

data_frame = pd.DataFrame(data)

schema = pa.DataFrameSchema({
    'state': pa.Column(pa.String),
    'year': pa.Column(pa.Int),
    'pop': pa.Column(pa.Float)
})

schema.validate(data_frame)

,state,year,pop
0,Ohio,2000,1.5
1,Ohion,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


You can select to display only top 5 entries by using **pd.head()** command

In [21]:
data_frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohion,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


To check the column of the dataframe, you can use **dataframe.columns**

In [22]:
data_frame.columns

Index(['state', 'year', 'pop'], dtype='object')

A column of the dataframe can be accessed using **dataframe['column_name']** or **dataframe.column_name**

In [24]:
data_frame['pop']

0    1.5
1    1.7
2    3.6
3    2.4
4    2.9
5    3.2
Name: pop, dtype: float64

In [25]:
data_frame.year

0    2000
1    2001
2    2002
3    2001
4    2002
5    2003
Name: year, dtype: int64

New column can be added to the dataframe using **dataframe['column_name'] = [data]**

In [27]:
data_frame['debt'] = [False, True, True, False, False, True]
data_frame

,state,year,pop,debt
0,Ohio,2000,1.5,False
1,Ohion,2001,1.7,True
2,Ohio,2002,3.6,True
3,Nevada,2001,2.4,False
4,Nevada,2002,2.9,False
5,Nevada,2003,3.2,True


Values of a column can also be changed

In [28]:
data_frame['debt'] = np.arange(6)
data_frame

,state,year,pop,debt
0,Ohio,2000,1.5,0
1,Ohion,2001,1.7,1
2,Ohio,2002,3.6,2
3,Nevada,2001,2.4,3
4,Nevada,2002,2.9,4
5,Nevada,2003,3.2,5


A column can be removed using **del dataframe['column_name']**

In [29]:
del data_frame['debt']
data_frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohion,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2
